In [ ]:
# !pip3 -q install requests beautifulsoup4 lxml openpyxl tqdm ipywidgets pyphen syllapy

In [255]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import trange
import nltk
from nltk import sent_tokenize,word_tokenize,wordpunct_tokenize,RegexpTokenizer
from nltk.corpus import stopwords
import syllapy as sp
import os
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('cmudict')

In [ ]:
df = pd.read_excel('../Input.xlsx')
err_url_id_new= []
titles, contents = [],[]
for i in trange(df.shape[0], desc='Progress: '):
    try:
        resp = re.get(df.iloc[i].URL)
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text,'html.parser')
            title, content = soup.find('h1', class_='entry-title'),soup.find('div', class_='td-post-content tagdiv-type')
            if not (title and content):
                title, content = soup.find('h1', class_='tdb-title-text'),soup.find('div', class_='td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type').find('div', class_='tdb-block-inner td-fix-index')
            title, content = title.get_text(), ' '.join(content.get_text().strip('\n').split('\n')[:-1])
            titles.append(title)
            contents.append(content)
        else:
            titles.append(resp.status_code)
            contents.append(resp.status_code)
    except Exception as e:
        err_url_id_new.append(df.iloc[i].URL_ID)
        print(f'Error: {e}, in URL: {df.iloc[i].URL} (ID: {df.iloc[i].URL_ID})')

df['TITLE'] = titles
df['CONTENT'] = contents
df.to_csv('Dataframe.csv',index=False)

In [257]:
sw_nltk = stopwords.words('english')


with open('../StopWords/StopWords_Auditor.txt','r') as StopWords_Auditor:
    sw_auditor = [i.split('|')[0].strip().lower() for i in StopWords_Auditor.readlines()]

with open('../StopWords/StopWords_DatesandNumbers.txt','r') as StopWords_DatesandNumbers:
    sw_datesandnumbers = [i.split('|')[0].strip().lower() for i in StopWords_DatesandNumbers.readlines()]

with open('../StopWords/StopWords_Generic.txt','r') as StopWords_Generic:
    sw_generic = [i.split('|')[0].strip().lower() for i in StopWords_Generic.readlines()]

with open('../StopWords/StopWords_GenericLong.txt','r') as StopWords_GenericLong:
    sw_genericlong = [i.split('|')[0].strip().lower() for i in StopWords_GenericLong.readlines()]

with open('../StopWords/StopWords_Geographic.txt','r') as StopWords_Geographic:
    sw_geographic = [i.split('|')[0].strip().lower() for i in StopWords_Geographic.readlines()]

with open('../StopWords/StopWords_Names.txt','r') as StopWords_Names:
    sw_names = [i.split('|')[0].strip().lower() for i in StopWords_Names.readlines()]

with open('../StopWords/StopWords_Currencies.txt', 'r') as StopWords_Names:
    sw_currencies = [i.split('|')[0].strip().lower() for i in StopWords_Names.readlines()]

pw_all = ["i", "you", "he", "she", "it", "we", "they", "them", "us", "him", "her", "his", "hers", "its", "theirs", "our", "your"]
sw_all = set(sw_auditor + sw_datesandnumbers + sw_generic + sw_genericlong + sw_geographic + sw_names + sw_names + sw_currencies + sw_nltk).difference(set(pw_all))



with open('../MasterDictionary/positive-words.txt') as positive_words_txt:
    positive_words = [i.strip('\n') for i in positive_words_txt.readlines()]

with open('../MasterDictionary/negative-words.txt') as negative_words_txt:
    negative_words = [i.strip('\n') for i in negative_words_txt.readlines()]

In [263]:
PATH = '../StopWords'
sw_x = []
for FILE in os.listdir(PATH):
    with open(os.path.join(PATH,FILE), 'r') as F:
        sw_x += [i.split('|')[0].strip().lower() for i in F.readlines()]
sw_x = set(sw_x + sw_nltk).difference(set(pw_all))
sw_x == sw_all

True

In [153]:
df =  pd.read_csv('Dataframe.csv')
df

,URL_ID,URL,TITLE,CONTENT
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"Telemedicine, the use of technology to diagnos..."
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,"The rise of e-health, or the use of electronic..."
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,2020 was the year the world was ravaged by the...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"“More gains on quality, affordability and acce..."
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"“More gains on quality, affordability and acce..."
...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,Coronavirus: Impact on the Hospitality Industry,Before jumping on the topic I would like to gi...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,Coronavirus impact on energy markets,As the coronavirus spreads around the world an...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,What are the key policies that will mitigate t...,"From Alibaba to Ping An and Google to Ford, co..."
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,Marketing Drives Results With A Focus On Problems,"When the British ruled India, many Indians acc..."


In [181]:
# collection = set()
# for i in trange(df.shape[0], desc='Progress: '):
#     collection.add(df.TITLE.loc[i][-1]) 
    # if 'US' in RegexpTokenizer(r'\w+').tokenize(df.TITLE.loc[i]+'. '+df.CONTENT.loc[i]):
#         print(i)
    # print('tokenised',i)
    # sw_filtered_text = [j for j in tokenized_text if j.lower not in sw_all]
    # print('filtered', i)
    # print(set([i.lower() for i in tokenized_text]).intersection(sw_all))
    # print()
    # print(set([i.lower() for i in tokenized_text]).intersection(sw_nltk))
    # print()
    # print()
# print(collection)

In [203]:
title = df.TITLE.loc[15]
content = df.CONTENT.loc[15]
if title[-1] in ['.','?','','!']:
    text = title +' '+content
else:
    text = title +'. '+content
text

'Oil prices by the year 2040, and how it will impact the world economy. We are in an interconnected world. Any change in one part of the world will always lead to some #changes in other parts of the world as well, maybe a bit later but surely there will be some change and that is what we are seeing in today’s world. Electric vehicles are the change that we are seeing in today’s world. With so many advancements in technology, economies are getting bigger. China might surpass the US and become a #superpower, we will have India inching towards a 15 trillion economy by 2040 and there will be competition among themselves to outshine the other. We are also seeing a paradigm shift toward a more multipolar world from a unipolar world where powers are moving towards certain clusters of the world and everyone wants to be a part of these clusters. But in all these rearrangements taking place. We are in a way creating a better and more sustainable life for our present as well as our future generat

In [204]:
tokenized_text = []
for i in sent_tokenize(text):
    tokens = RegexpTokenizer(r'\w+').tokenize(i)
    tokenized_text.append(tokens)
    # print(f'{tokens} :: {len(tokens)}')
# tokenized_text = RegexpTokenizer(r'\w+').tokenize(text)
# 'US' in tokenized_text
print(tokenized_text)

[['Oil', 'prices', 'by', 'the', 'year', '2040', 'and', 'how', 'it', 'will', 'impact', 'the', 'world', 'economy'], ['We', 'are', 'in', 'an', 'interconnected', 'world'], ['Any', 'change', 'in', 'one', 'part', 'of', 'the', 'world', 'will', 'always', 'lead', 'to', 'some', 'changes', 'in', 'other', 'parts', 'of', 'the', 'world', 'as', 'well', 'maybe', 'a', 'bit', 'later', 'but', 'surely', 'there', 'will', 'be', 'some', 'change', 'and', 'that', 'is', 'what', 'we', 'are', 'seeing', 'in', 'today', 's', 'world'], ['Electric', 'vehicles', 'are', 'the', 'change', 'that', 'we', 'are', 'seeing', 'in', 'today', 's', 'world'], ['With', 'so', 'many', 'advancements', 'in', 'technology', 'economies', 'are', 'getting', 'bigger'], ['China', 'might', 'surpass', 'the', 'US', 'and', 'become', 'a', 'superpower', 'we', 'will', 'have', 'India', 'inching', 'towards', 'a', '15', 'trillion', 'economy', 'by', '2040', 'and', 'there', 'will', 'be', 'competition', 'among', 'themselves', 'to', 'outshine', 'the', 'other

In [205]:
sw_filtered_text = []
for i in tokenized_text: 
    sw_filtered_sent = []
    for j in i:
        if j == 'US':
            sw_filtered_sent.append(j)
            continue
        if j.lower() not in sw_all:
            sw_filtered_sent.append(j)
    sw_filtered_text.append(sw_filtered_sent)
# sw_filtered_text = [[j for j in i if j.lower() not in sw_all] for i in tokenized_text]
print(sw_filtered_text)
# j=0
# for i in sw_filtered_text:
#     if 'US' in i:
#         print(i, j)
#         break
#     j+=1    

[['Oil', 'prices', '2040', 'it', 'impact', 'world', 'economy'], ['We', 'interconnected', 'world'], ['change', 'part', 'world', 'lead', 'parts', 'world', 'bit', 'surely', 'change', 'we', 'today', 'world'], ['Electric', 'vehicles', 'change', 'we', 'today', 'world'], ['advancements', 'technology', 'economies', 'bigger'], ['surpass', 'US', 'superpower', 'we', 'inching', '15', 'economy', '2040', 'competition', 'outshine'], ['We', 'paradigm', 'shift', 'multipolar', 'world', 'unipolar', 'world', 'moving', 'clusters', 'world', 'part', 'clusters'], ['rearrangements', 'taking'], ['We', 'creating', 'sustainable', 'life', 'our', 'present', 'our', 'future', 'generations'], ['We', 'witnessed', 'COP26', 'we', 'leaders', 'top', 'countries', 'taking', 'pledges', 'create', 'sustainable', 'future', 'answer', 'lies', 'safer', 'greener', 'planet', 'electric', 'vehicles', 'occupy', 'huge', 'chunk', 'it'], ['Internal', 'combustion', 'engines', 'peaked', 'electric', 'vehicles', 'cheaper', 'cheaper', 'governme

In [251]:
sentiment_words = {'positive':[], 'negative': []}
sent_count, word_count, char_count, pron_count, syll_count, comp_count = 0, 0, 0, 0, 0, 0
for sent in sw_filtered_text:
    sent_count += 1
    for word in sent:
        word_count += 1
        char_count += len(word)
        syll_count += sp.count(word)
        if word.lower() in positive_words:
            sentiment_words['positive'].append(word)
        if word.lower() in negative_words:
            sentiment_words['negative'].append(word)
        if word.lower() in pw_all:
            pron_count += 1
        if sp.count(word) > 2:
            comp_count += 1
print(sentiment_words)
print()
print ('sentences: ', sent_count)
print()
print('words ', word_count)
print()
print('letters: ', char_count)
print()
print('syllables: ', syll_count)
print()
print('pesonal-pronouns: ', pron_count)
print()
print('complex-words: ', comp_count)

{'positive': ['lead', 'surpass', 'outshine', 'sustainable', 'top', 'sustainable', 'cheaper', 'cheaper', 'work', 'properly', 'safe', 'efficient', 'affordable', 'improve', 'FAME', 'sufficient', 'sufficient', 'great', 'sufficient', 'boost', 'easy', 'usable', 'important'], 'negative': ['unipolar', 'lies', 'crude', 'lagging', 'lagging', 'Crude', 'shortage', 'crude', 'harmful', 'havoc', 'problems', 'Crude', 'spilling', 'problems', 'crude', 'waste', 'concern', 'waste', 'waste', 'toxic', 'negative', 'improper', 'waste', 'dumped', 'illegally', 'waste', 'contamination', 'contamination', 'waste']}

sentences:  53

words  589

letters:  3579

syllables:  1133

pesonal-pronouns:  57

complex-words:  175


In [213]:
set(positive_words).intersection(set(negative_words))

{'envious', 'enviously', 'enviousness'}

In [243]:
import syllapy
import pyphen

word = input("Word: ")

print(syllapy.count(word))
print(len(pyphen.Pyphen(lang='nl_NL').inserted(word).split('-')))

Word:  loved


1
2


In [242]:
pyphen.Pyphen(lang='nl_NL').inserted('passed')

'pas-sed'

In [223]:
dfo = pd.read_excel('../Output Data Structure.xlsx')
scores = {i:0 for i in dfo.columns[2:]}
# dfo
scores

{'POSITIVE SCORE': 0,
 'NEGATIVE SCORE': 0,
 'POLARITY SCORE': 0,
 'SUBJECTIVITY SCORE': 0,
 'AVG SENTENCE LENGTH': 0,
 'PERCENTAGE OF COMPLEX WORDS': 0,
 'FOG INDEX': 0,
 'AVG NUMBER OF WORDS PER SENTENCE': 0,
 'COMPLEX WORD COUNT': 0,
 'WORD COUNT': 0,
 'SYLLABLE PER WORD': 0,
 'PERSONAL PRONOUNS': 0,
 'AVG WORD LENGTH': 0}

In [253]:
scores['POSITIVE SCORE'] = len(sentiment_words['positive'])
scores['NEGATIVE SCORE'] = len(sentiment_words['negative'])
scores['POLARITY SCORE'] = (scores['POSITIVE SCORE'] - scores['NEGATIVE SCORE'])/ ((scores['POSITIVE SCORE'] + scores['NEGATIVE SCORE']) + 0.000001)
scores['SUBJECTIVITY SCORE'] = (scores['POSITIVE SCORE'] + scores['NEGATIVE SCORE'])/ ((word_count) + 0.000001)
scores['AVG SENTENCE LENGTH'] = word_count / sent_count
scores['PERCENTAGE OF COMPLEX WORDS'] = comp_count / word_count
scores['FOG INDEX'] = 0.4 * (scores['AVG SENTENCE LENGTH'] + scores['PERCENTAGE OF COMPLEX WORDS'])
scores['AVG NUMBER OF WORDS PER SENTENCE'] = scores['AVG SENTENCE LENGTH']
scores['COMPLEX WORD COUNT'] = comp_count
scores['WORD COUNT'] = word_count
scores['SYLLABLE PER WORD'] = syll_count / word_count
scores['PERSONAL PRONOUNS'] = pron_count
scores['AVG WORD LENGTH'] = char_count / word_count
scores.keys()

dict_keys(['POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'])

In [230]:
scores['AVG SENTENCE LENGTH'] = word_count / sent_count
scores['PERCENTAGE OF COMPLEX WORDS'] = comp_count / word_count
scores['FOG INDEX'] = 0.4 * (scores['AVG SENTENCE LENGTH'] + scores['PERCENTAGE OF COMPLEX WORDS'])

0.08828522905214732

In [278]:
dfo

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
